In [1]:
import cobra
from cobra import Metabolite, Gene, Reaction
from cobra.flux_analysis import single_gene_deletion
import pandas as pd
from medusa.test import load_universal_modelseed
import medusa
from pickle import load
from medusa.flux_analysis import deletion

In [2]:
with open("../results/ensembles/pto_ensemble_100_genes_added_e3.pickle", 'rb') as infile:
    ensemble = load(infile)
#universal = load_universal_modelseed()

In [3]:
#set media conditions for each ensemble

#complete media has all of the exchange reactions open
complete = {}
for reaction in ensemble.base_model.reactions:
    if "EX_" in reaction.id:
        complete[reaction.id] = 1000

        
#for the other medias, use teh base biolog salts (minus heme) and add the media types needed
from medusa.test import load_biolog_plata
biolog_base_composition, biolog_base_dict, biolog_thresholded = load_biolog_plata()
biolog_base_composition = biolog_base_composition.drop(18)

gluc_mm = {}

for x in biolog_base_composition.ID:
    gluc_mm['EX_' + x] = 1000 
#gluc_mm['EX_cpd00027_e'] = 1000

glut_mm = {}

for x in biolog_base_composition.ID:
    glut_mm['EX_' + x] = 1000
glut_mm['EX_cpd00053_e'] = 1000

#load in the anderson metabolites
anderson_df = pd.read_csv("../data/metabolite_ids_Anderson2014.csv")
anderson_media_list = []
met_ids = [met for met in anderson_df.met_id]
for met in met_ids:
    met="EX_" + str(met) + "_e"
    anderson_media_list.append(met)
    
    
anderson = {}
for x in biolog_base_composition.ID:
    anderson['EX_' + x] = 1000 
for y in anderson_media_list:
    if y in ensemble.base_model.reactions:
        anderson[y] = 1000
        #print (y + " is IN the model")
    else:
        pass
        #print (y+ " is not in the base model") 

leu_mm = {}
for x in biolog_base_composition.ID:
    leu_mm['EX_' + x] = 1000
leu_mm['EX_cpd00107_e'] = 1000

hrp_mm = {}
for x in biolog_base_composition.ID:
    hrp_mm['EX_' + x] = 1000
hrp_mm['EX_cpd00082_e'] = 1000

ile_mm = {}
for x in biolog_base_composition.ID:
    ile_mm['EX_' + x] = 1000
ile_mm['EX_cpd00322_e'] = 1000


val_mm = {}
for x in biolog_base_composition.ID:
    val_mm['EX_' + x] = 1000
val_mm['EX_cpd00156_e'] = 1000
mock = {}
##add seedling media here

flg22 = {}
##add seedling media here

In [4]:
model_list = []
for model in ensemble.members:
    model_list.append(model.id)

In [5]:
ensemble.base_model.medium = val_mm
filevar = 'val_100_e3'
file_path = '../results/gene_essentiality/ensemble_essentialiity/'
file_suf_xl = '_media_ensemble_deletions.xlsx'
file_name_ensemble_deletions = file_path + filevar + file_suf_xl
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter(file_name_ensemble_deletions) as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        #print (key)
        value.to_excel(writer, sheet_name=key)
reps = {}
for i in range(0,100):
    gene_knockouts = pd.read_excel(file_name_ensemble_deletions ,sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98], reps[99]])


file_suf_csv = '_media_ensemble_deletions.csv'
file_name_concat =file_path + filevar + "concat" + file_suf_csv
file_name_unique =file_path + filevar + "unique" + file_suf_csv


all_biolog.to_csv(file_name_concat)
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv(file_name_unique)

print ("I have completed: " + filevar)
#manually determine the essential genes using count if function, save as same name excel extention

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [9]:
filevar = 'val_100_e3'
import re
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]

complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        #print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)
print (filevar + " is ready for R.")

val_100_e3 is ready for R.


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [26]:
filevar = 'all_genes_100_e3'
file_name = file_path + filevar + "_orthologies.csv"
kegg_list_super = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/kegg_list_super.xlsx', names = ['Gene', 'Pathway'])
kegg_list_super_dict = pd.Series(kegg_list_super.Pathway.values,index=kegg_list_super.Gene).to_dict()
complete = pd.read_csv((file_name), names = ['Gene', 'Pathway'])
complete_p_count = complete.groupby(['Pathway']).count()
file_name = file_path + filevar + "_orthologies_counted.csv"
complete_p_count.to_csv(file_name)
complete_p_count.reset_index(level=0, inplace=True)
complete_p_count['supergroup'] = complete_p_count['Pathway'].map(kegg_list_super_dict)
file_name = file_path + filevar + "_supergrouped_orthologies.csv"
complete_p_count.to_csv(file_name)

In [116]:
file_name = file_path + filevar + "unique" + file_suf_xl
complete = pd.read_excel(file_name)
essen = complete.query('counts_of==0')['ids']
file_name = file_path + filevar + "essential_only" + file_suf_csv
essen.to_csv(file_name)
#for row in complete.itertuples():
    #print (complete.counts_of)
    #x = str(complete.counts_of)
    #print (x)
    #if "0" is x:
     #   print ('yes')
        


/Users/phil/cobrapython/lib/python3.7/site-packages/ipykernel_launcher.py:5 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [117]:
patric_path = file_path + "PATRIC_id_mapping.csv"
patric = pd.read_csv(patric_path)
patric_dict = patric.to_dict(orient = 'index')
complete = pd.read_csv(file_name, names = ["ids"])
complete_ids = [ids for ids in complete.ids]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass

        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)
file_suf_r = "for_r.csv"
file_name = file_name_unique =file_path + filevar + "unique" + file_suf_r
unique_list = []

#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list

for z in complete_and_patric_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv(file_name)

#after r, use the next cell

fig|223283.9.peg.1 PSPTO_0001
fig|223283.9.peg.1290 PSPTO_1268
fig|223283.9.peg.1494 PSPTO_1473
fig|223283.9.peg.1497 PSPTO_1476
fig|223283.9.peg.1522 PSPTO_1501
fig|223283.9.peg.1560 PSPTO_1534
fig|223283.9.peg.1575 PSPTO_1549
fig|223283.9.peg.1577 PSPTO_1551
fig|223283.9.peg.1663 PSPTO_1639
fig|223283.9.peg.1773 PSPTO_1745
fig|223283.9.peg.184 PSPTO_0178
fig|223283.9.peg.1841 PSPTO_1812
fig|223283.9.peg.1842 PSPTO_1813
fig|223283.9.peg.1871 PSPTO_1842
fig|223283.9.peg.190 PSPTO_0184
fig|223283.9.peg.191 PSPTO_0185
fig|223283.9.peg.2 PSPTO_0002
fig|223283.9.peg.2197 PSPTO_2166
fig|223283.9.peg.2413 PSPTO_2378
fig|223283.9.peg.2415 PSPTO_2380
fig|223283.9.peg.2416 PSPTO_2381
fig|223283.9.peg.2417 PSPTO_2382
fig|223283.9.peg.2418 PSPTO_2383
fig|223283.9.peg.3424 PSPTO_3345
fig|223283.9.peg.3599 PSPTO_3514
fig|223283.9.peg.3735 PSPTO_3646
fig|223283.9.peg.3748 PSPTO_3658
fig|223283.9.peg.3813 PSPTO_3721
fig|223283.9.peg.3834 PSPTO_3742
fig|223283.9.peg.3923 PSPTO_3826
fig|223283.9.peg.39

In [115]:
#print (complete_gene_list)
complete_list_mod = [] 
for z in complete_gene_list:
    z = "fig|" + z
    complete_list_mod.append(z)
#print (complete_list_mod)
new_pat=pd.Series(patric.Target.values,index=patric.Source).to_dict()
patric_genes_essential = []
for key,value in new_pat.items():
    if key in complete_list_mod:
        print (key, value)
        patric_genes_essential.append(value)
complete_and_patric_gene_list = patric_genes_essential + complete_gene_list


fig|223283.9.peg.1 PSPTO_0001
fig|223283.9.peg.1290 PSPTO_1268
fig|223283.9.peg.1494 PSPTO_1473
fig|223283.9.peg.1497 PSPTO_1476
fig|223283.9.peg.1522 PSPTO_1501
fig|223283.9.peg.1560 PSPTO_1534
fig|223283.9.peg.1575 PSPTO_1549
fig|223283.9.peg.1577 PSPTO_1551
fig|223283.9.peg.1663 PSPTO_1639
fig|223283.9.peg.1773 PSPTO_1745
fig|223283.9.peg.184 PSPTO_0178
fig|223283.9.peg.1841 PSPTO_1812
fig|223283.9.peg.1842 PSPTO_1813
fig|223283.9.peg.1871 PSPTO_1842
fig|223283.9.peg.190 PSPTO_0184
fig|223283.9.peg.191 PSPTO_0185
fig|223283.9.peg.2 PSPTO_0002
fig|223283.9.peg.2197 PSPTO_2166
fig|223283.9.peg.2413 PSPTO_2378
fig|223283.9.peg.2415 PSPTO_2380
fig|223283.9.peg.2416 PSPTO_2381
fig|223283.9.peg.2417 PSPTO_2382
fig|223283.9.peg.2418 PSPTO_2383
fig|223283.9.peg.3424 PSPTO_3345
fig|223283.9.peg.3599 PSPTO_3514
fig|223283.9.peg.3735 PSPTO_3646
fig|223283.9.peg.3748 PSPTO_3658
fig|223283.9.peg.3813 PSPTO_3721
fig|223283.9.peg.3834 PSPTO_3742
fig|223283.9.peg.3923 PSPTO_3826
fig|223283.9.peg.39

In [33]:
ensemble.base_model.medium = glut_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/glut_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [38]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/glut_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/glut_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_glut.csv')

In [41]:
ensemble.base_model.medium = gluc_mm
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/gluc_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        value.to_excel(writer, sheet_name=key)


performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [43]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/gluc_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/gluc_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_gluc.csv')

In [42]:
ensemble.base_model.medium = anderson
deletion_results = {}
with ensemble.base_model:
    for model in model_list:
        print('performing deletions for ' + model)
        ensemble.set_state(model)
        deletion_result = single_gene_deletion(ensemble.base_model)
        deletion_results[model] = deletion_result

with pd.ExcelWriter('../results/gene_essentiality/ensemble_essentialiity/anderson_media_ensemble_deletions.xlsx') as writer:  # doctest: +SKIP
    for key, value in deletion_results.items():
        print (key)
        value.to_excel(writer, sheet_name=key)

performing deletions for PST_gapfilled_0
performing deletions for PST_gapfilled_1
performing deletions for PST_gapfilled_2
performing deletions for PST_gapfilled_3
performing deletions for PST_gapfilled_4
performing deletions for PST_gapfilled_5
performing deletions for PST_gapfilled_6
performing deletions for PST_gapfilled_7
performing deletions for PST_gapfilled_8
performing deletions for PST_gapfilled_9
performing deletions for PST_gapfilled_10
performing deletions for PST_gapfilled_11
performing deletions for PST_gapfilled_12
performing deletions for PST_gapfilled_13
performing deletions for PST_gapfilled_14
performing deletions for PST_gapfilled_15
performing deletions for PST_gapfilled_16
performing deletions for PST_gapfilled_17
performing deletions for PST_gapfilled_18
performing deletions for PST_gapfilled_19
performing deletions for PST_gapfilled_20
performing deletions for PST_gapfilled_21
performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_23
pe

In [44]:
reps = {}
for i in range(0,99):
    gene_knockouts = pd.read_excel('../results/gene_essentiality/ensemble_essentialiity/anderson_media_ensemble_deletions.xlsx',sheet_name="PST_gapfilled_"+str(i+1))
    gene_knockouts.columns = [col for col in gene_knockouts.columns]
    reps[i] = gene_knockouts
all_biolog = pd.concat([reps[0], reps[1], reps[2], reps[3], reps[4], reps[5], reps[6], reps[7], reps[8], reps[9], reps[10], reps[11], reps[12], reps[13], reps[14], reps[15]
                        , reps[16], reps[17], reps[18], reps[19], reps[20], reps[21], reps[22], reps[23], reps[24], reps[25], reps[26], reps[27], reps[28], reps[29], reps[30]
                        , reps[31], reps[32], reps[33], reps[34], reps[35], reps[36], reps[37], reps[38], reps[39], reps[40], reps[41], reps[42], reps[43], reps[44], reps[45]
                        , reps[46], reps[47], reps[48], reps[49], reps[50], reps[51], reps[52], reps[53], reps[54], reps[55], reps[56], reps[57], reps[58], reps[59], reps[60]
                        , reps[61], reps[62], reps[63], reps[64], reps[65], reps[66], reps[67], reps[68], reps[69], reps[70], reps[71], reps[72], reps[73], reps[74], reps[75]
                        , reps[76], reps[77], reps[78], reps[79], reps[80], reps[81], reps[82], reps[83], reps[84], reps[85], reps[86], reps[87], reps[88], reps[89], reps[90]
                        , reps[91], reps[92], reps[93], reps[94], reps[95], reps[96], reps[97], reps[98]])

all_biolog.to_csv('../results/gene_essentiality/ensemble_essentialiity/anderson_essentiality.csv')
df3 = all_biolog['growth'].groupby(all_biolog['ids']).unique().apply(pd.Series)
df3.to_csv('../results/gene_essentiality/ensemble_essentialiity/unique_anderson.csv')

In [6]:
model = ensemble.extract_member("PST_gapfilled_24")

In [7]:
cobra.io.write_sbml_model(model, "pst_24.xml")

Error encountered trying to <set model history>.
LibSBML error code -5: The object passed as an argument to the method is not of a type that is valid for the operation or kind of object involved. For example, handing an invalidly-constructed ASTNode to a method expecting an ASTNode will result in this error.


In [9]:
model2 = cobra.io.read_sbml_model("pst_24.xml")

In [46]:
import re
complete = pd.read_excel("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/ensemble_1_essen_compile.xlsx")
complete_ids = [ids for ids in complete.complete]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_complete_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.anderson]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_anderson_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.glut]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_glut_essential_all_for_r.csv")
complete_ids = [ids for ids in complete.gluc]
for y in complete_ids:
    if "nan":
        pass
    else:
        print (y)
complete_gene_list=[]
for x in complete_ids:
    x = str(x)
    if 'nan' in x:
        pass
    else:
        result = re.split("frozenset\\({'", x)
    for y in result:
        if "frozenset" in y:
            pass
        else:
            complete_gene_list.append(str(y[:-3]))
pd.DataFrame(complete_gene_list)

#change this for r, so that the keggrest can use it
unique_list = []
for z in complete_gene_list:
    if z not in unique_list:
        if 'PSPTO_' in z:
            z = "pst:" + z
            unique_list.append(z)
        else: pass
    else:
        pass
listo = pd.DataFrame(unique_list)
listo.to_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/unique_gluc_essential_all_for_r.csv")

In [48]:
#map genes to patric identifiers

patric = pd.read_csv("../results/gene_essentiality/ensemble_essentialiity/ensemble_1_updated_genes/PATRIC_id_mapping.csv")

In [56]:
pat_dict = {}
for x in patric.Source:
    pat_dict[x] = patric.Target

In [57]:
pat_dict

{'fig|223283.9.peg.1': 0      PSPTO_0001
 1      PSPTO_1005
 2      PSPTO_1041
 3      PSPTO_1051
 4      PSPTO_1053
 5      PSPTO_0103
 6      PSPTO_1077
 7      PSPTO_1079
 8      PSPTO_1080
 9      PSPTO_1081
 10     PSPTO_1104
 11     PSPTO_1105
 12     PSPTO_1108
 13     PSPTO_1112
 14     PSPTO_1128
 15     PSPTO_1129
 16     PSPTO_1131
 17     PSPTO_1136
 18     PSPTO_0112
 19     PSPTO_1169
 20     PSPTO_1178
 21     PSPTO_1182
 22     PSPTO_1196
 23     PSPTO_0118
 24     PSPTO_1215
 25     PSPTO_1232
 26     PSPTO_1243
 27     PSPTO_1268
 28     PSPTO_1271
 29     PSPTO_1276
           ...    
 688    PSPTO_0800
 689    PSPTO_0802
 690    PSPTO_0805
 691    PSPTO_0806
 692    PSPTO_0809
 693    PSPTO_0817
 694    PSPTO_0080
 695    PSPTO_0836
 696    PSPTO_0083
 697    PSPTO_0085
 698    PSPTO_0885
 699    PSPTO_0886
 700    PSPTO_0089
 701    PSPTO_0923
 702    PSPTO_0090
 703    PSPTO_0948
 704    PSPTO_0092
 705    PSPTO_0954
 706    PSPTO_0955
 707    PSPTO_0956
 708    P

In [25]:
filevars = ['biolog_base', 'complete' , 'ile' , 'hmm' , 'glut' , 'gluc' , 'val' , 'anderson']
filesuff = '_100_e3_supergrouped_orthologies.csv'
filepre = '../results/gene_essentiality/ensemble_essentialiity/'
for x in filevars:  
    filevar = x
    filename = filepre + x + filesuff
    table = pd.read_csv(filename)
    

 